In [1]:
import matplotlib.pyplot as plt
import uproot
import numpy as np
import pandas as pd

In [2]:
nfs_path = "/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/appliedNNdata/thirdRun/"

In [3]:
Ups4S_file = uproot.open(nfs_path + "Ups4S_NN_predicted.root:variables;1")
df_Ups4S = Ups4S_file.arrays(library="pd")

In [6]:
FSPs_file = uproot.open(nfs_path + "FSPs.root:variables;1")
df_FSPs = FSPs_file.arrays(library="pd")

## save df's

In [7]:
df_FSPs.to_csv(nfs_path + "FSPs_df.csv")

In [5]:
df_Ups4S.to_csv(nfs_path + "Ups4S_df.csv")

## load df's

In [ ]:
df_Ups4S = pd.read_csv(nfs_path + "Ups4S_df.csv")
df_FSPs = pd.read_csv(nfs_path + "FSPs_df.csv")

# Analysis